In [1]:
!pip install -q bitsandbytes datasets accelerate loralib evaluate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

# 1. Data Preprocessing 

In [3]:
import json
from datasets import Dataset, Features, ClassLabel, Value, Sequence

def merge_columns(example):
    example["prediction"] = example["questions"] + " ->: " + example["answers"]
    return example

f = open("/kaggle/input/ecommerce-faq-chatbot-dataset/Ecommerce_FAQ_Chatbot_dataset.json")
data = json.load(f)

questions = []
answers = []
for i in data["questions"]:
    questions += [i["question"]]
    answers += [i["answer"]]
    
dataset = Dataset.from_dict({
    "id": list(range(len(questions))),
    "questions": questions,
    "answers": answers
    },
    features = Features({
        "id": Value(dtype='string'),
        "questions": Value(dtype = "string"),
        "answers": Value(dtype = "string")
    }
))

dataset = dataset.map(merge_columns)

  0%|          | 0/79 [00:00<?, ?ex/s]

In [4]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
import evaluate
import warnings
warnings.filterwarnings("ignore")
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

def get_model(model_id):
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        load_in_8bit=True,
        device_map='auto',
    )

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    return model, tokenizer

def predict(question, model, tokenizer):    
    batch = tokenizer(f"{question} ->: ", return_tensors='pt')
    with torch.cuda.amp.autocast():
      output_tokens = model.generate(**batch, max_new_tokens=50)
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)


def evaluate_bleu(model, tokenizer):
    predictions = []
    for i in dataset["questions"]:
        predictions.append(predict(i, model, tokenizer))
    bleu = evaluate.load('bleu')
    references = dataset["prediction"]
    result = bleu.compute(predictions = predictions, references = references)
    return result

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 2. GPT2

In [5]:
gpt2_model, gpt2_tokenizer = get_model("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
gpt2_result = evaluate_bleu(gpt2_model, gpt2_tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [7]:
gpt2_result

{'bleu': 0.2767918012600496,
 'precisions': [0.39582013932868904,
  0.2788750536711035,
  0.2421926184756497,
  0.21955555555555556],
 'brevity_penalty': 1.0,
 'length_ratio': 1.3242941012021248,
 'translation_length': 4737,
 'reference_length': 3577}

# 3. distilGPT2

In [8]:
distilgpt2_model, distilgpt2_tokenizer = get_model("distilgpt2")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
distilgpt2_result = evaluate_bleu(distilgpt2_model, distilgpt2_tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [11]:
distilgpt2_result

{'bleu': 0.3117019365297296,
 'precisions': [0.43157894736842106,
  0.375548801080716,
  0.35392088827203333,
  0.33357117374241885],
 'brevity_penalty': 0.8380774696900191,
 'length_ratio': 0.849874196253844,
 'translation_length': 3040,
 'reference_length': 3577}

# 4. GPT2_XL

In [12]:
gpt2_xl_model, gpt2_xl_tokenizer = get_model("openai-community/gpt2-xl")

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [13]:
gpt2_xl_result = evaluate_bleu(gpt2_xl_model, gpt2_xl_tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [14]:
gpt2_xl_result

{'bleu': 0.2225315462681065,
 'precisions': [0.3038288637420555,
  0.22426239799121156,
  0.1987923089146671,
  0.18104280656581911],
 'brevity_penalty': 1.0,
 'length_ratio': 1.8034665921162987,
 'translation_length': 6451,
 'reference_length': 3577}

# 5. falcon-7b

In [17]:
falcon_model, falcon_tokenizer = get_model("tiiuae/falcon-7b")

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [18]:
falcon_result = evaluate_bleu(falcon_model, falcon_tokenizer)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

In [19]:
falcon_result

{'bleu': 0.31618571309489846,
 'precisions': [0.580952380952381,
  0.4891437935272429,
  0.44919559695173583,
  0.4191852825229961],
 'brevity_penalty': 0.657411947240959,
 'length_ratio': 0.7045009784735812,
 'translation_length': 2520,
 'reference_length': 3577}